In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.stats.multicomp import MultiComparison
from scipy.stats import spearmanr, pearsonr
pd.set_option('display.width', 2000)

In [ ]:
nucleus_df = pd.read_csv('csv/nuclei_g4_2.csv')
image_df = pd.read_csv('csv/images_g4_2.csv')
print(nucleus_df['Location'].unique())


In [ ]:

#nucleus_df = nucleus_df[nucleus_df['CellType'].isin(['neunPositive', 'neunPositiveLow'])]
nucleus_df['CellType'] = nucleus_df['CellType'].replace('neunPositiveLow', 'neunPositive')

merged_df = pd.merge(nucleus_df, image_df[['ImageName','g4Background']], on='ImageName')
print(merged_df)
# Assuming 'df' is your DataFrame
print(len(nucleus_df[nucleus_df['CellType']=='neunPositiveLow']))
print(nucleus_df['Location'].unique())

In [ ]:
order =['Sham', 'Contra', 'Ipsi']

mean_ch3_intensity = nucleus_df.groupby(['Condition', 'ImageName', 'CellType'])['Ch3Intensity'].mean().reset_index()
#mean_ch3_intensity = mean_ch3_intensity.drop([8,9,10,11])
merged_ch3 = merged_df.groupby(['Condition','ImageName', 'CellType', 'g4Background'])['Ch3Intensity'].mean().reset_index()
merged_ch3['BackgroundSubtracted'] = merged_ch3['Ch3Intensity'] - merged_ch3['g4Background']

palette = sns.color_palette(['royalblue'], 2)
merged_ch3 = merged_ch3.drop([8,9,10,11])
print(merged_ch3)
boxplot = sns.boxplot(data=merged_ch3, showfliers = False, x='Condition', y='BackgroundSubtracted', hue='CellType',order=order, palette='Set3')
stripplot = sns.stripplot(data=merged_ch3, x='Condition', y='BackgroundSubtracted', order=order, dodge=True, hue='CellType', palette=palette, alpha=0.7)
handles, labels = boxplot.get_legend_handles_labels()

# Create custom legend
plt.legend(handles=handles[:2], labels=labels[:2])
plt.ylim([0,75])
plt.ylabel('CTCF fluorescence')
plt.savefig('plots/G4NeuronsBackgroundSubtracted.pdf')
plt.show()

In [ ]:
from scipy.stats import f_oneway
from statsmodels.stats.multicomp import pairwise_tukeyhsd

print(mean_ch3_intensity.keys())

neun_positive_df = merged_ch3[merged_ch3['CellType'] == 'neunPositive']

# Perform ANOVA test on filtered data
anova_results = f_oneway(neun_positive_df[neun_positive_df['Condition'] == 'Contra']['Ch3Intensity'],
                         neun_positive_df[neun_positive_df['Condition'] == 'Ipsi']['Ch3Intensity'],
                         neun_positive_df[neun_positive_df['Condition'] == 'Sham']['Ch3Intensity'])

# Perform Tukey HSD post hoc test
tukey_results = pairwise_tukeyhsd(neun_positive_df['Ch3Intensity'], neun_positive_df['Condition'])

print("ANOVA Results:")
print("F-statistic:", anova_results.statistic)
print("p-value:", anova_results.pvalue)

print("\nTukey HSD Results:")
print(tukey_results)